Importing libraries

In [ ]:
import re
import random
import numpy as np

loading movie_lines.txt and movie_conversation.txt

In [ ]:
lines = open('movie_lines.txt',encoding = 'iso-8859-1').read().split('\n')
conversations = open('movie_conversations.txt').read().split('\n')

In [ ]:
id2line = {}
for i in range(len(lines)):
    temp_line = lines[i].split(' +++$+++ ')
    id2line[temp_line[0]] = temp_line[-1]
del temp_line,i    
    
conversations_ids = []
for i in range(len(conversations)):
    conversations_ids.append(conversations[i].split(' +++$+++ ')[-1][1:-1].replace("'","").split(', '))
del i 

In [ ]:
questions = []
answers = []
for conversation_id in conversations_ids:
    x = len(conversation_id)
    if (x%2 != 0):
        x -= 1
    for i in range(0,x,2):
        questions.append(id2line[conversation_id[i]])
        answers.append(id2line[conversation_id[i+1]])
    

Cleaning texts

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"he's","he is",text)
    text = re.sub(r"she's","she is",text)
    text = re.sub(r"that's","that is",text)
    text = re.sub(r"what's","what is",text)
    text = re.sub(r"where's","where is",text)
    text = re.sub(r"'ll"," will",text)
    text = re.sub(r"'ve"," have",text)
    text = re.sub(r"'re"," are",text)
    text = re.sub(r"'d"," would",text)
    text = re.sub(r"won't","will not",text)
    text = re.sub(r"can't","cannot",text)
    text = re.sub('[^a-zA-Z0-9]',' ',text)
    return text

In [ ]:
clean_questions = []    
for question in questions:
    clean_questions.append(clean_text(question))
    
clean_answers = []    
for answer in answers:
    clean_answers.append(clean_text(answer))

Combining both question and answer

In [ ]:
pairs = list(zip(clean_questions,clean_answers))

tokanization with START and END

In [ ]:
import numpy as np

input_docs = []
target_docs = []
input_tokens = set()
target_tokens = set()
for line in pairs[:400]:
  input_doc, target_doc = line[0], line[1]
  # Appending each input sentence to input_docs
  input_docs.append(input_doc)
  # Splitting words from punctuation  
  target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))
  # Redefine target_doc below and append it to target_docs
  target_doc = '<START> ' + target_doc + ' <END>'
  target_docs.append(target_doc)
  
  # Now we split up each sentence into words and add each unique word to our vocabulary set
  for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
    if token not in input_tokens:
      input_tokens.add(token)
  for token in target_doc.split():
    if token not in target_tokens:
      target_tokens.add(token)
input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

input_features_dict = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_features_dict = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

reverse_input_features_dict = dict(
    (i, token) for token, i in input_features_dict.items())
reverse_target_features_dict = dict(
    (i, token) for token, i in target_features_dict.items())


max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])

encoder_input_data = np.zeros(
    (len(input_docs), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):
    for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
        #Assign 1. for the current line, timestep, & word in encoder_input_data
        encoder_input_data[line, timestep, input_features_dict[token]] = 1.
    
    for timestep, token in enumerate(target_doc.split()):
        decoder_input_data[line, timestep, target_features_dict[token]] = 1.
        if timestep > 0:
            decoder_target_data[line, timestep - 1, target_features_dict[token]] = 1.

In [ ]:
print(pairs[:5])
print(input_docs[:5])

[('can we make this quick   roxanne korrine and andrew barrett are having an incredibly horrendous public break  up on the quad   again ', 'well  i thought we would start with pronunciation  if that is okay with you '), ('not the hacking and gagging and spitting part   please ', 'okay    then how  bout we try out some french cuisine   saturday   night '), ('you are asking me out   that is so cute  what is your name again ', 'forget it '), ('no  no  it s my fault    we didn t have a proper introduction    ', 'cameron '), ('the thing is  cameron    i am at the mercy of a particularly hideous breed of loser   my sister   i cannot date until she does ', 'seems like she could get a date easy enough   ')]
['can we make this quick   roxanne korrine and andrew barrett are having an incredibly horrendous public break  up on the quad   again ', 'not the hacking and gagging and spitting part   please ', 'you are asking me out   that is so cute  what is your name again ', 'no  no  it s my fault   

Implementing LSTM encoder and decoder and saving the model


In [ ]:
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model
#Dimensionality
dimensionality = 256
#The batch size and number of epochs
batch_size = 10
epochs = 600
#Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(dimensionality, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]
#Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

#Model
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
#Compiling
training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')
#Training
training_model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)
training_model.save('training_model.h5')

Epoch 1/600
32/32 [==============================] - 11s 102ms/step - loss: 0.9745 - accuracy: 0.0185 - val_loss: 1.0600 - val_accuracy: 0.0213
Epoch 2/600
32/32 [==============================] - 2s 72ms/step - loss: 0.9273 - accuracy: 0.0210 - val_loss: 1.0657 - val_accuracy: 0.0222
Epoch 3/600
32/32 [==============================] - 2s 76ms/step - loss: 0.9218 - accuracy: 0.0209 - val_loss: 1.0723 - val_accuracy: 0.0222
Epoch 4/600
32/32 [==============================] - 2s 73ms/step - loss: 0.9201 - accuracy: 0.0210 - val_loss: 1.0818 - val_accuracy: 0.0222
Epoch 5/600
32/32 [==============================] - 2s 73ms/step - loss: 0.9186 - accuracy: 0.0204 - val_loss: 1.0953 - val_accuracy: 0.0213
Epoch 6/600
32/32 [==============================] - 2s 73ms/step - loss: 0.9178 - accuracy: 0.0206 - val_loss: 1.1007 - val_accuracy: 0.0213
Epoch 7/600
32/32 [==============================] - 2s 76ms/step - loss: 0.9370 - accuracy: 0.0194 - val_loss: 1.1116 - val_accuracy: 0.0213
Epoc

Load saved model and implementation of decoding of response from model

In [ ]:
from keras.models import load_model
training_model = load_model('training_model.h5')
encoder_inputs = training_model.input[0]
encoder_outputs, state_h_enc, state_c_enc = training_model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

latent_dim = 256
decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]
decoder_outputs, state_hidden, state_cell = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_hidden, state_cell]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

def decode_response(test_input):
    #Getting the output states to pass into the decoder
    states_value = encoder_model.predict(test_input)
    #Generating empty target sequence of length 1
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    #Setting the first token of target sequence with the start token
    target_seq[0, 0, target_features_dict['<START>']] = 1.
    
    #A variable to store our response word by word
    decoded_sentence = ''
    
    stop_condition = False
    while not stop_condition:
      #Predicting output tokens with probabilities and states
      output_tokens, hidden_state, cell_state = decoder_model.predict([target_seq] + states_value)
#Choosing the one with highest probability
      sampled_token_index = np.argmax(output_tokens[0, -1, :])
      sampled_token = reverse_target_features_dict[sampled_token_index]
      decoded_sentence += " " + sampled_token
#Stop if hit max length or found the stop token
      if (sampled_token == '<END>' or len(decoded_sentence) > max_decoder_seq_length):
        stop_condition = True
#Update the target sequence
      target_seq = np.zeros((1, 1, num_decoder_tokens))
      target_seq[0, 0, sampled_token_index] = 1.
      #Update states
      states_value = [hidden_state, cell_state]
    return decoded_sentence

Chatbot simulation

In [ ]:
class ChatBot:
  negative_responses = ("no", "nope", "nah", "naw", "not a chance", "sorry")
  exit_commands = ("quit", "pause", "exit", "goodbye", "bye", "later", "stop")
#Method to start the conversation
  def start_chat(self):
    user_response = input("Hi, I'm a chatbot trained on random dialogs. Would you like to chat with me?\n")
    
    if user_response in self.negative_responses:
      print("Ok, have a great day!")
      return
    self.chat(user_response)
#Method to handle the conversation
  def chat(self, reply):
    while not self.make_exit(reply):
      reply = input(self.generate_response(reply)+"\n")
    
  #Method to convert user input into a matrix
  def string_to_matrix(self, user_input):
    tokens = re.findall(r"[\w']+|[^\s\w]", user_input)
    user_input_matrix = np.zeros(
      (1, max_encoder_seq_length, num_encoder_tokens),
      dtype='float32')
    for timestep, token in enumerate(tokens):
      if token in input_features_dict:
        user_input_matrix[0, timestep, input_features_dict[token]] = 1.
    return user_input_matrix
  
  #Method that will create a response using seq2seq model we built
  def generate_response(self, user_input):
    input_matrix = self.string_to_matrix(user_input)
    chatbot_response = decode_response(input_matrix)
    #Remove <START> and <END> tokens from chatbot_response
    chatbot_response = chatbot_response.replace("<START>",'')
    chatbot_response = chatbot_response.replace("<END>",'')
    return chatbot_response
#Method to check for exit commands
  def make_exit(self, reply):
    for exit_command in self.exit_commands:
      if exit_command in reply:
        print("Ok, have a great day!")
        return True
    return False
  
chatbot = ChatBot()
chatbot.start_chat()

Hi, I'm a chatbot trained on random dialogs. Would you like to chat with me?
hi there
 hi there how are you today 
i m fine
 everything is fine on this side 
nice
 how is work going today 
good
 is the name in english of one of the main characters
bye
Ok, have a great day!


In [ ]:
chatbot.start_chat()

Hi, I'm a chatbot trained on random dialogs. Would you like to chat with me?
yes sure
 i don really really that that that expect expect expect
then what do you want
 that is what i i you 
what
 no thirty 
are you mad
 thirty are 
lol
 no thirty 
bye
Ok, have a great day!
